# Change the background

Distribution:
- 249 - blur
- 249 - bright
- 249 - flip
- 249 - rot
- 400 - mix of above

In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import cv2
from PIL import Image

from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize
from skimage import io, transform, color, data_dir, draw, img_as_ubyte
from data_process import label_2_colour, resize_2_256, result_2_image

In [ ]:
# Read all the new images with different background from the original from its folder
new_bk_path = "../data/carseg_data/new_background/"
new_bk_names = glob.glob(new_bk_path + "*.png")
new_bk_names.sort()
len(new_bk_names)

In [ ]:
def combine_image_and_labels(img_path, label_path):
    
    org_arr = np.load(label_path)
    aug_arr = cv2.imread(img_path)
    aug_arr = cv2.cvtColor(aug_arr, cv2.COLOR_BGR2RGB) # cv2 reads bgr default
    new_arr = np.zeros_like(org_arr)
    new_arr[:,:,0:3] = aug_arr[:,:,0:3] # take the rgb info from the new image
    new_arr[:,:,3] = org_arr[:,:,3]     # take the label info from the original image

    return new_arr

def find_corresponding_label(img_path):
    """Function to find the corresponding label for the image

    Args:
        img_path (str): path to the image
        all_labels (list): list of all the labels

    Returns:
        str: path to the corresponding label
    """
    img_name = img_path.split("/")[-1]
    label_name = img_name.split(".")[0][:-3] + ".npy"   # Removes the _bk from name and adds .npy
    label_path = os.path.join("../data/carseg_data/augmented_arrays", label_name)
    return label_path

# Relate the image with new background to its corresponding label
org_names = glob.glob("../data/carseg_data/augmented_arrays/*.npy")
org_names.sort()

for img in new_bk_names:
    name = img.split("/")[-1].split(".")[0] 
    lab = find_corresponding_label(img)
    new_arr = combine_image_and_labels(img, lab)
    np.save(os.path.join("../data/carseg_data/new_background_arrays/", "{}.npy".format(name)), new_arr)


In [ ]:
# Visualise the new images with new background side by side with labels
import time
new_arrs = glob.glob("../data/new_dataset/*.npy")[:100]
new_arrs.sort()
new_arrs = np.array([np.load(arr) for arr in new_arrs])
for i in range(len(new_arrs)):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.imshow(new_arrs[i][:,:,0:3])
    plt.subplot(1,2,2)
    plt.imshow(label_2_colour(new_arrs[i][:,:,3]))
    plt.show()
    time.sleep(0.001)
    clear_output(wait=True)

In [ ]:
# Function to randomly choose a given number of images from each directory
import random
import shutil
def pick_random_imgs(dir_path, dest_path, n):
    """Picks n random images from a directory and copies them to a destination directory

    Args:
        dir_path (str): string path to the directory
        dest_path (str): string path to the destination directory
        n (int): number of images to pick
    """
    categories = ["rot", "blur", "bright", "flip"]
    for cat in categories:
        files = glob.glob(os.path.join(dir_path, cat + "*.png"))
        selected_files = random.sample(files, n)
        for file in selected_files:
            shutil.copy(file, dest_path)
    

In [ ]:
# Copy the images to a new directory
copy_from_here = "../data/carseg_data/augmented_images/"
copy_to_here = "../data/carseg_data/upload_us/"
pick_random_imgs(copy_from_here, copy_to_here, 350)

In [ ]:
# display the image
img = cv2.imread(copy_to_here + "rot_911.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
